# Importação de Bibliotecas e Leitura Dataset

In [3]:
import pandas as pd
import spacy # Biblioteca de Processamento de Linguagem Natural
import re # Módulo Python para trabalhar com Regex
import contractions # Biblioteca para expansão de contrações
import unidecode # Biblioteca para transliteração de caracteres Unicode em ASCII
from collections import Counter

In [4]:
# Descrição do Dataset de Review de Filmes no IMDB em língua inglesa
df = pd.read_csv("../../../data/raw/IMDB Dataset.csv")
print("Dimensões do dataframe: ", df.shape)
print("Colunas do dataframe: ", df.columns)
print("Tipo da coluna Rewiew: ", type(df['review'].iloc[0]))
print("Tipo da coluna Sentiment: ", type(df['sentiment'].iloc[0]))
print("Número de linhas duplicadas:", df.duplicated().sum())
print("Número de valores ausentes: ", df.isnull().sum())
df.head()

Dimensões do dataframe:  (50000, 2)
Colunas do dataframe:  Index(['review', 'sentiment'], dtype='object')
Tipo da coluna Rewiew:  <class 'str'>
Tipo da coluna Sentiment:  <class 'str'>
Número de linhas duplicadas: 418
Número de valores ausentes:  review       0
sentiment    0
dtype: int64


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Pré-processamento
- Remoção das reviews duplicadas
- Remoção de Tags HTML e contrações
- Colocar tudo em letras minúsculas
- Remoção de pontuações, números e acentos
- Remoção de stopwords
- Lematização
- Tokenização

In [5]:
# 49.582 reviews em 50.000 linhas -> tem 418 duplicadas
df = df.drop_duplicates(subset='review').reset_index(drop=True)
print(df.info(), '\n')
print(df.describe(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49582 entries, 0 to 49581
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     49582 non-null  object
 1   sentiment  49582 non-null  object
dtypes: object(2)
memory usage: 774.8+ KB
None 

                                                   review sentiment
count                                               49582     49582
unique                                              49582         2
top     One of the other reviewers has mentioned that ...  positive
freq                                                    1     24884 



In [6]:
# Carregar modelo SpaCy em Inglês
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# Função de pré-processamento
def preprocess_text(text):
    # Remove tags HTML
    text = re.sub(r"<.*?>", "", text)
    # Remove contrações
    text = contractions.fix(text)
    # Normaliza acentos
    text = unidecode.unidecode(text)  
    # Processa o texto com SpaCy
    doc = nlp(text)
    # Lematiza, coloca lowercase, remove stop words, pontuação e simbolos que não são letras
    tokens = [
        token.lemma_.lower() for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    return " ".join(tokens), tokens

df[["clean_review", "tokens"]] = df["review"].apply(lambda x: pd.Series(preprocess_text(x)))

In [7]:
# Calcular a frequência de todas as palavras no corpus
all_tokens = [token for tokens_list in df['tokens'] for token in tokens_list]
token_freq = Counter(all_tokens)

# Definir um limite mínimo de frequência
min_freq = 50

# Função para filtrar tokens com frequência mínima
def filter_low_freq_tokens(tokens_list):
    return [token for token in tokens_list if token_freq[token] >= min_freq]

# Aplicar a filtragem à coluna 'tokens'
df['tokens'] = df['tokens'].apply(filter_low_freq_tokens)

# Atualizar a coluna 'clean_review' com os tokens filtrados
df['clean_review'] = df['tokens'].apply(lambda x: ' '.join(x))

In [8]:
# Salvar o novo arquivo
df.to_csv("../../../data/processed/IMDB_cleaned_SpaCy.csv", index=False)

print("Pré-processamento com tokenização finalizado e salvo em 'IMDB_cleaned_SpaCy.csv'")

Pré-processamento com tokenização finalizado e salvo em 'IMDB_cleaned_SpaCy.csv'


# CSV processado
Esse CSV gerado a partir do pré-processamento possui duas colunas principais:
- Coluna "clean_review": formato com apenas texto, que podem ser usados para conversão de vetores numéricos como Bag of Words, TF-IDF e n-grams.
- Coluna "tokens': formato com apenas sequências de tokens, que são ideais para embeddings e treinamento de modelos de deep learning

In [9]:
df_processado = pd.read_csv("../../../data/processed/IMDB_cleaned_SpaCy.csv")
print("Dimensões do dataframe: ", df_processado.shape)
print("Colunas do dataframe: ", df_processado.columns)
df_processado.head()

Dimensões do dataframe:  (49582, 4)
Colunas do dataframe:  Index(['review', 'sentiment', 'clean_review', 'tokens'], dtype='object')


,review,sentiment,clean_review,tokens
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch oz episode hook right e...,"['reviewer', 'mention', 'watch', 'oz', 'episod..."
1,A wonderful little production. <br /><br />The...,positive,wonderful little production film technique old...,"['wonderful', 'little', 'production', 'film', ..."
2,I thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...,"['think', 'wonderful', 'way', 'spend', 'time',..."
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...,"['basically', 'family', 'little', 'boy', 'jake..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,mattei love time money visually stunning film ...,"['mattei', 'love', 'time', 'money', 'visually'..."
